In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.AI, 1.0.0-beta.23558.10"

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23557.4"

In [3]:
#!value --name key
YOUR AZURE OPEN AI KEY

In [4]:
#!value --name endpoint
https://your-enpoint.openai.azure.com/

In [5]:
#!connect azure-openai --model-type TextEmbeddingGenerator --kernel-name knowledge --api-key @value:key --endpoint @value:endpoint --deployment text-embedding-ada-002

Kernel added: #!knowledge(duckDbVectorRAW)

Kernel added: #!knowledge(duckDbVector)

Kernel added: #!knowledge

In [6]:
#!connect azure-openai --model-type ChatCompletion --kernel-name chat --api-key @value:key --endpoint @value:endpoint --deployment gtp-35-turbo --use-knowledge knowledge

Kernel added: #!chat(skill)

Kernel added: #!chat(text)

Kernel added: #!chat

### we will be using text embeddings and cosine distance similarity to focus the conversation with the agent on the questions the user is about ot submit

In [7]:
#r "nuget: Microsoft.ML,  3.0.0-preview.23511.1"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json Installed Packages Microsoft.ML, 3.0.0-preview.23511.1

In [8]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

## Focus the conversation

At each turn let's filter out irrelevant turns for the question at hand. TextEmbedding ans cosine similarity will help us keep the conversation on track!

In [9]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using Microsoft.DotNet.Interactive.AI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI.Tokenizers;
using Microsoft.SemanticKernel.Skills.Core;
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.Embeddings.VectorOperations;

var chatCompletionKernel = Microsoft.DotNet.Interactive.Kernel.Root.FindKernelByName("chat") as ChatCompletionKernel;
var knowledgeKernel = Microsoft.DotNet.Interactive.Kernel.Root.FindKernelByName("knowledge") as KnowledgeKernel;

chatCompletionKernel.SetUserTurnFilter(async (turn, userPrompt, token) =>{
    var userEmbeddings = await knowledgeKernel.GenerateEmbeddingsAsync(userPrompt, token);
    var questionRelevance = userEmbeddings
                    .CosineSimilarity(turn.QuestionTexEmbedding);
    var answerRelevance = userEmbeddings
                    .CosineSimilarity(turn.AnswerTexEmbedding);
    var relevanceThreshold = 0.8;
    if (questionRelevance >= relevanceThreshold || answerRelevance >= relevanceThreshold)
    {
        return true;
    }else{
        //"The turn is not relevant to the conversation.".Display();
        //turn.Display();
        return false;
    }
});

## what are we talking about??

Let's tap into ML and use K-Means to cluster the chatlog using the text embeddings of each turns!

In [10]:
public class ModelInput
{
  public string Text {get;set;}
  [VectorType(1536)]
  public float[] Embedding {get;set;}
}

In [11]:
IDataView clusteredData = null;
ClusteringPredictionTransformer<KMeansModelParameters> model = null;

chatCompletionKernel.OnConversationTurnCompleted(( chatLog, toekn) =>
{
   var embeddings = new List<ModelInput>();
   var ctx = new MLContext();
   var clusterCount  = Math.Min(20, 2+ (chatLog.Turns.Count / 5));
   if(chatLog.Turns.Count < clusterCount)
   {
      return Task.CompletedTask;
   }  
   foreach (var turn in chatLog.Turns)
   {
      switch (turn){
       case ChatLog.ChatUserTurn  userTurn:
          embeddings.Add(new ModelInput() { Text = userTurn.Question, Embedding = userTurn.QuestionTexEmbedding});
          embeddings.Add(new ModelInput() { Text = userTurn.Answer, Embedding = userTurn.AnswerTexEmbedding });
          break;
          case ChatLog.ChatSystemTurn systemTurn:
         // embeddings.Add(new ModelInput() { Text = systemTurn.Prompt, Embedding = systemTurn..AnswerTexEmbedding.AsReadOnlySpan().ToArray() });
          break;
      }
   }
   var idv = ctx.Data.LoadFromEnumerable(embeddings);

   var pipeline = 
   ctx.Clustering.Trainers.KMeans("Embedding", numberOfClusters: clusterCount);
   model = pipeline.Fit(idv);
   clusteredData = model.Transform(idv);
   return Task.CompletedTask;
});

## Start chatting then!

In [12]:
I would like to ask you questions about coding in C#

Sure, I'll do my best to answer your questions about coding in C#. What would you like to know?

In [13]:
How do I cook a fish?

As an AI language model, I don't eat or cook food, but here are some general steps to cook a fish:

1. Clean the fish thoroughly, removing scales, guts, and other unwanted parts.
2. Season the fish with salt, pepper, and any other desired spices or herbs.
3. Heat a pan or grill to medium-high heat.
4. Add oil or butter to the pan or grill.
5. Place the fish on the pan or grill, skin-side down if applicable.
6. Cook the fish for 3-5 minutes on each side, or until the flesh is opaque and flakes easily with a fork.
7. Serve the fish with your desired sides and garnishes.

In [14]:
What is a c# class?

In C#, a class is a blueprint or a template for creating objects that share common properties, methods, and events. It defines the structure and behavior of an object, including its data members (fields) and member functions (methods). 

A class can be thought of as a user-defined data type that encapsulates data and behavior. It provides a way to organize and modularize code, making it easier to maintain and reuse. 

For example, if you were creating a program to manage a library, you might define a class called "Book" that contains properties such as "Title", "Author", and "ISBN", as well as methods for checking out and returning books. You could then create multiple instances of the "Book" class to represent different books in the library.

In [15]:
What is the most common modal scale in heavy metal music?

The most common modal scale in heavy metal music is the Phrygian mode.

In [16]:
How do I setup my amplifier to play heavy metal guitar riffs?

To set up your amplifier to play heavy metal guitar riffs, follow these steps:

1. Start by selecting a high-gain channel on your amplifier. This will give you the distortion and overdrive necessary for heavy metal riffs.

2. Adjust the gain knob to your desired level. For heavy metal, you'll want a high gain setting to get that crunchy, distorted sound.

3. Adjust the bass, mid, and treble knobs to your liking. For heavy metal, you'll typically want to boost the bass and treble while cutting the mids to get a more aggressive sound.

4. If your amplifier has a presence knob, adjust it to add more clarity and definition to your sound.

5. Experiment with the volume and master volume knobs to find the right balance between volume and distortion.

6. Finally, consider adding some effects like delay, reverb, or chorus to enhance your sound and add depth to your riffs.

Remember, the settings that work best for heavy metal guitar riffs will depend on your personal preferences and the specif

In [29]:
What is the mest way to model embeddings in a relational database?

There are several ways to model embeddings in a relational database, but one common approach is to use a table with columns for each dimension of the embedding. For example, if the embedding has 100 dimensions, the table would have 100 columns. Each row in the table would represent an entity or object that has an embedding, and the values in the columns would represent the values of the embedding dimensions for that entity.

Another approach is to use a separate table for each dimension of the embedding, with each table containing a single column for the values of that dimension. This approach can be more flexible and scalable, as it allows for adding or removing dimensions without having to modify the main table. However, it can also be more complex to query and join the data from multiple tables.

Ultimately, the best way to model embeddings in a relational database depends on the specific requirements and constraints of the application, as well as the size and complexity of the embe

In [30]:
What is a motorbike?

A motorbike, also known as a motorcycle or bike, is a two-wheeled vehicle powered by an engine. It typically has a seat for the rider, handlebars for steering, and footrests for the rider's feet. Motorbikes come in various sizes and styles, including sport bikes, cruisers, touring bikes, and dirt bikes. They are commonly used for transportation, recreation, and racing.

In [34]:
Then, what is the best way to represent a motorbike in C# using classes and interfaces? I am thinking of making a motorbike racing game in .NET.

If you are making a motorbike racing game in .NET, the best way to represent a motorbike in C# using classes and interfaces would be to create a base class for the motorbike and then create derived classes for each specific type of motorbike. Here's an example:

```
public abstract class Motorbike
{
    // Fields
    protected string make;
    protected string model;
    protected int year;
    protected float engineSize;
    protected bool isRunning;
    protected float speed;
    protected float acceleration;
    protected float braking;

    // Constructor
    public Motorbike(string make, string model, int year, float engineSize, float acceleration, float braking)
    {
        this.make = make;
        this.model = model;
        this.year = year;
        this.engineSize = engineSize;
        this.isRunning = false;
        this.speed = 0f;
        this.acceleration = acceleration;
        this.braking = braking;
    }

    // Properties
    public string Make
    {
        get { 

In [35]:
Show me an example of a class that represents a motorbike using C#, not sure what methods I would need for the game simulation of the bike.

Sure, here's an example of a class that represents a motorbike using C#:

```
public class Motorbike
{
    // Fields
    private string make;
    private string model;
    private int year;
    private float engineSize;
    private bool isRunning;
    private float speed;
    private float acceleration;
    private float braking;

    // Constructor
    public Motorbike(string make, string model, int year, float engineSize, float acceleration, float braking)
    {
        this.make = make;
        this.model = model;
        this.year = year;
        this.engineSize = engineSize;
        this.isRunning = false;
        this.speed = 0f;
        this.acceleration = acceleration;
        this.braking = braking;
    }

    // Properties
    public string Make
    {
        get { return make; }
        set { make = value; }
    }

    public string Model
    {
        get { return model; }
        set { model = value; }
    }

    public int Year
    {
        get { return year; }
        s

In [36]:
clusteredData.Preview().Display();

4 columns, 32 rows Schema [ Text: String, Embedding: Vector<Single, 1536>, PredictedLabel: Key<UInt32, 0-4> {ScoreColumnKind, ScoreValueKind, ScoreColumnSetId}, Score: Vector<Single, 5> {ScoreColumnKind, ScoreColumnSetId, ScoreValueKind} ] Count 4 (values) index value 0 Text: String Name Text Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) 1 Embedding: Vector<Single, 1536> Name Embedding Index 1 IsHidden False Type Vector<Single, 1536> Dimensions [ 1536 ] IsKnownSize True ItemType Single RawType System.Single Size 1536 RawType Microsoft.ML.Data.VBuffer<System.Single> Annotations Schema [ ] Count 0 (values) (empty) 2 PredictedLabel: Key<UInt32, 0-4> {ScoreColumnKind, ScoreValueKind, ScoreColumnSetId} Name PredictedLabel Index 2 IsHidden False Type Key<UInt32, 0-4> Count 5 RawType System.UInt32 Annotations ScoreColumnKind, ScoreValueKind, ScoreColumnSetId Schema [ ScoreColumnKind: String, ScoreValueKind: String, ScoreColumnSetId: Key<UInt32, 0-2147483646> ] Count 3 (values) index value 0 ScoreColumnKind: String Name ScoreColumnKind Index 0 IsHidden False Type String Annotations 1 ScoreValueKind: String Name ScoreValueKind Index 1 IsHidden False Type String Annotations 2 ScoreColumnSetId: Key<UInt32, 0-2147483646> Name ScoreColumnSetId Index 2 IsHidden False Type Key Annotations 3 Score: Vector<Single, 5> {ScoreColumnKind, ScoreColumnSetId, ScoreValueKind} Name Score Index 3 IsHidden False Type Vector<Single, 5> Dimensions [ 5 ] IsKnownSize True ItemType Single RawType System.Single Size 5 RawType Microsoft.ML.Data.VBuffer<System.Single> Annotations ScoreColumnKind, ScoreColumnSetId, ScoreValueKind Schema [ ScoreColumnKind: String, ScoreColumnSetId: Key<UInt32, 0-2147483646>, ScoreValueKind: String ] Count 3 (values) index value 0 ScoreColumnKind: String Name ScoreColumnKind Index 0 IsHidden False Type String Annotations 1 ScoreColumnSetId: Key<UInt32, 0-2147483646> Name ScoreColumnSetId Index 1 IsHidden False Type Key Annotations 2 ScoreValueKind: String Name ScoreValueKind Index 2 IsHidden False Type String Annotations ColumnView index value 0 Text: String Column Text: String Name Text Index 0 IsHidden False Type String RawType System.ReadOnlyMemory<System.Char> Annotations Schema [ ] Count 0 (values) (empty) Values index value 0 I would like to ask you questions about coding in C# 1 Sure, I'll do my best to answer your questions about coding in C#. What would you like to know? 2 How do I cook a fish? 3 As an AI language model, I don't eat or cook food, but here are some general steps to cook a fish:

1. Clean the fish thoroughly, removing scales, guts, and other unwanted parts.
2. Season the fish with salt, pepper, and any other desired spices or herbs.
3. Heat a pan or grill to medium-high heat.
4. Add oil or butter to the pan or grill.
5. Place the fish on the pan or grill, skin-side down if applicable.
6. Cook the fish for 3-5 minutes on each side, or until the flesh is opaque and flakes easily with a fork.
7. Serve the fish with your desired sides and garnishes. 4 What is a c# class? 5 In C#, a class is a blueprint or a template for creating objects that share common properties, methods, and events. It defines the structure and behavior of an object, including its data members (fields) and member functions (methods). 

A class can be thought of as a user-defined data type that encapsulates data and behavior. It provides a way to organize and modularize code, making it easier to maintain and reuse. 

For example, if you were creating a program to manage a library, you might define a class called "Book" that contains properties such as "Title", "Author", and "ISBN", as well as methods for checking out and returning books. You could then create multiple instances of the "Book" class to represent different books in the library. 6 What is the most common modal scale in heavy metal music? 7 The most common modal scale in heavy metal music is the Phrygi

## Lets find what has been covered in the conversation.
Now that we have the `KMean` model we can look at the centroids, they represent the topic that turns had in common. 

In [37]:
VBuffer<float>[] centroids = default;

In [38]:
model.Model.GetClusterCentroids(ref centroids, out var _);

In [ ]:
centroids.Display();

now using `Microsoft.DotNet.Interactive.AIUtilities` and the extension `ScoreBySimilarityTo` we will try to collect the top 3 questions and 3 answer that are very close to each topic.

In [39]:
using Microsoft.DotNet.Interactive.AIUtilities;

var log = chatCompletionKernel.ChatLog;

var examples = centroids.Select(c => {
    var embedding = c.GetValues().ToArray();
    var questions = log.Turns
        .OfType<ChatLog.ChatUserTurn>()
        .ScoreBySimilarityTo(embedding, new CosineSimilarityComparer<float[]>(v => v), turn => turn.QuestionTexEmbedding )
        .OrderByDescending(e => e.Value)
        .Where(e => e.Value > 0.8)
        .Take(3)
        .Select(e => e.Key.Question)
        .ToArray();

    var answers = log.Turns
        .OfType<ChatLog.ChatUserTurn>()
        .ScoreBySimilarityTo(embedding, new CosineSimilarityComparer<float[]>(v => v), turn => turn.AnswerTexEmbedding )
        .OrderByDescending(e => e.Value)
        .Where(e => e.Value > 0.8)
        .Take(3)
        .Select(e => e.Key.Answer)
        .ToArray();
   return new {
        CenstroidEmbedding = embedding,
        Text = questions.Concat(answers).ToArray()
        };
    }
).ToArray();


In [40]:
examples.Display();

index value 0 { CenstroidEmbedding = System.Single[], Text = System.String[] } CenstroidEmbedding [ -0.0045551048, 0.006455068, -0.004881875, -0.02412982, -0.018067542, 0.0007876146, -0.004960471, -0.0052060722, -0.010661278, -0.043034513, 0.0052415035, 0.021151206, -0.003764674, -0.002854415, -0.0026767273, 0.01568986, 0.025198301, -0.0018648285, 0.012310172, -0.001751964 ... (1516 more) ] Text [ Then, what is the best way to represent a motorbike in C# using classes and interfaces? I am thinking of making a motorbike racing game in .NET., Then, what is the best way to represent a motorbike in C# using classes and interfaces? I am thinking of making a motorbike racing game in .NET., Then, what is the best way to represent a motorbike in C# using classes and interfaces?, The best way to represent a motorbike in C# using classes and interfaces would be to create a base class for the motorbike and then create derived classes for each specific type of motorbike. Here's an example:

```
public abstract class Motorbike
{
 // Fields
 protected string make;
 protected string model;
 protected int year;
 protected float engineSize;
 protected bool isRunning;

 // Constructor
 public Motorbike(string make, string model, int year, float engineSize)
 {
 this.make = make;
 this.model = model;
 this.year = year;
 this.engineSize = engineSize;
 this.isRunning = false;
 }

 // Properties
 public string Make
 {
 get { return make; }
 set { make = value; }
 }

 public string Model
 {
 get { return model; }
 set { model = value; }
 }

 public int Year
 {
 get { return year; }
 set { year = value; }
 }

 public float EngineSize
 {
 get { return engineSize; }
 set { engineSize = value; }
 }

 public bool IsRunning
 {
 get { return isRunning; }
 }

 // Methods
 public virtual void Start()
 {
 isRunning = true;
 Console.WriteLine("The {0} {1} has started.", make, model);
 }

 public virtual void Stop()
 {
 isRunning = false;
 Console.WriteLine("The {0} {1} has stopped.", make, model);
 }

 public abstract void Accelerate();

 public abstract void Brake();
}

public class SportBike : Motorbike
{
 // Constructor
 public SportBike(string make, string model, int year, float engineSize) : base(make, model, year, engineSize)
 {
 }

 // Methods
 public override void Accelerate()
 {
 if (isRunning)
 {
 Console.WriteLine("The {0} {1} is accelerating quickly.", make, model);
 }
 else
 {
 Console.WriteLine("The {0} {1} is not running.", make, model);
 }
 }

 public override void Brake()
 {
 if (isRunning)
 {
 Console.WriteLine("The {0} {1} is braking quickly.", make, model);
 }
 else
 {
 Console.WriteLine("The {0} {1} is not running.", make, model);
 }
 }
}

public class Cruiser : Motorbike
{
 // Constructor
 public Cruiser(string make, string model, int year, float engineSize) : base(make, model, year, engineSize)
 {
 }

 // Methods
 public override void Accelerate()
 {
 if (isRunning)
 {
 Console.WriteLine("The {0} {1} is accelerating smoothly.", make, model);
 }
 else
 {
 Console.WriteLine("The {0} {1} is not running.", make, model);
 }
 }

 public override void Brake()
 {
 if (isRunning)
 {
 Console.WriteLine("The {0} {1} is braking smoothly.", make, model);
 }
 else
 {
 Console.WriteLine("The {0} {1} is not running.", make, model);
 }
 }
}
```

In this example, the `Motorbike` class is an abstract base class that defines the common fields and methods for all types of motorbikes. The `SportBike` and `Cruiser` classes are derived classes that inherit from the `Motorbike` class and implement the `Accelerate` and `Brake` methods in their own specific ways. 

Using this approach, you can easily add new types of motorbikes by creating new derived classes that inherit from the `Motorbike` class and implement their own unique behavior. Additionally, you can use interfaces to define common behavior for all motorbikes, such as `IMotorbike` interface that defines methods for starting, stopping, accelerating, and braking., Sure, here's an example of a class that rep

Using the `TextCompletion` kernel we will try to generate a label for each centroid using the examples we collected

In [41]:
var textCompletionKernel = Microsoft.DotNet.Interactive.Kernel.Root.FindKernelByName("chat(text)") as TextCompletionKernel;

In [42]:
using Microsoft.DotNet.Interactive.Events;
public record CentroidLabel(string Label, float[] Embedding);
var labels = new List<CentroidLabel>();
foreach (var example in examples)
{
    textCompletionKernel.SetSuppressDisplay(true);
    var result = await textCompletionKernel.SendAsync(
        new SubmitCode(
"""
Give the following conversation examples, please provide a label that classifies the topic being discussed. Return a single respose and that should be a single label, not a list. Thhe label will be used to classify future conversations. make sure not to returl a list, or bulelt points, only a single line response.

The response should be in like : 

Topic : Prorgramming


Here are the conversation examples :

"""+ string.Join("\n", example.Text.Select(t => $"- {t}"))
));
    textCompletionKernel.SetSuppressDisplay(false);
    var response = result.Events.OfType<ReturnValueProduced>().Last().FormattedValues.First(fm => fm.MimeType == "text/plain").Value;
    labels.Add(new CentroidLabel(response, example.CenstroidEmbedding));
}

And this is the resul!

In [43]:
labels.Display();

index value 0 CentroidLabel { Label = Topic: Programming, Embedding = System.Single[] } Label Topic: Programming Embedding [ -0.0045551048, 0.006455068, -0.004881875, -0.02412982, -0.018067542, 0.0007876146, -0.004960471, -0.0052060722, -0.010661278, -0.043034513, 0.0052415035, 0.021151206, -0.003764674, -0.002854415, -0.0026767273, 0.01568986, 0.025198301, -0.0018648285, 0.012310172, -0.001751964 ... (1516 more) ] 1 CentroidLabel { Label = Topic: Database modeling, Embedding = System.Single[] } Label Topic: Database modeling Embedding [ -0.019690322, 0.0074574924, 0.0125079015, -0.03223809, -0.013383181, 0.014590225, -0.0023842067, -0.011381577, -0.01262123, -0.037007995, -0.009087111, 0.015590968, 0.0020191246, 0.015170503, -0.010306402, 0.023493309, 0.02167419, 0.016822029, 0.02860094, -0.024521086 ... (1516 more) ] 2 CentroidLabel { Label = Topic: Heavy Metal Music, Embedding = System.Single[] } Label Topic: Heavy Metal Music Embedding [ 0.019309083, -0.019269338, 0.011363486, -0.03139314, -0.020371001, 0.019811884, -0.031179953, -0.022803467, 0.0018026272, -0.012925929, 0.009142992, 0.0033195533, -0.008282289, 0.012882266, -0.016962307, 0.012483177, 0.030233376, 0.007203494, 0.005841951, 0.0005095892 ... (1516 more) ] 3 CentroidLabel { Label = Topic: Cooking, Embedding = System.Single[] } Label Topic: Cooking Embedding [ 0.025924776, 0.009006391, -0.0034855865, 0.0031638218, -0.013990513, 0.001340036, -0.03153697, -0.015431978, -0.013966864, -0.03650188, 0.0111081395, 0.016827773, -0.020932883, -0.009962637, -0.01634736, 0.019313686, 0.03336887, 0.020132894, 0.021527618, -0.01802596 ... (1516 more) ] 4 CentroidLabel { Label = Topic: Music, Embedding = System.Single[] } Label Topic: Music Embedding [ -0.004719562, -0.016547356, -0.012674535, -0.018823992, -0.0060666567, 0.03347228, -0.009583241, -0.036992565, -0.02607778, -0.0056306776, 0.0031455639, -0.005547706, 0.0018215985, 0.005364747, -0.0014557588, 0.008105663, 0.018891498, 0.01604237, 0.0009886855, -0.015773943 ... (1516 more) ]

In [ ]:
chatCompletionKernel.ResetChatLog();